네이버영화평점 분석
==
- ### 감성분석
- ### 네이버 영화평점 데이터
    - ### Naver sentiment movie corpus v1.0
    - ### https://github.com/e9t/nsmc
        - #### 영화 리뷰 20만 건
        - #### 평가 데이터 : {0, 1}, 부정(0), 긍정(1)
- ### 한글 자연어 처리
    - ### konlpy 패키지 제공 (https://konlpy-ko.readthedocs.io/ko/v0.4.3/api/konlpy/)
        - #### Twitter 문서(형태소) 분석 라이브러리 사용

# 로지스틱회귀를 이용한 감성분석

In [1]:
!pip install konlpy

## 네이버 영화 평점 데이터 다운로드

In [1]:
# 네이버 영화 평점 데이터 다운로드
!curl -L https://bit.ly/2X9Owwr -o ratings_train.txt
!curl -L https://bit.ly/2WuLd5I -o ratings_test.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100   152  100   152    0     0    107      0  0:00:01  0:00:01 --:--:--   107

  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0

  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0

  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
 23 14.0M   23 3406k    0     0   839k      0  0:00:17  0:00:04  0:00:13 4280k
 66 14.0M   66 9566k    0     0  1881k      0  0:00:07  0:00:05  0:00:02 5239k
100 14.0M  100 14.0M    0     0  2460k      0  0

In [4]:
import konlpy
import pandas as pd
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import make_pipeline
# import pickle
# import os.path

### 데이터 읽기

#### pd.read_csv(...., keep_default_na=False)
- If keep_default_na is False, and na_values are not specified, 
- no strings will be parsed as NaN.

In [5]:
# 데이터 로드
df_train = pd.read_csv('ratings_train.txt', delimiter='\t', keep_default_na=False)
df_test = pd.read_csv('ratings_test.txt', delimiter='\t', keep_default_na=False)

In [9]:
type(df_train)

pandas.core.frame.DataFrame

In [7]:
df_train.shape, df_test.shape

((150000, 3), (50000, 3))

In [6]:
df_train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


### 훈련 데이터, 시험 데이터 구성

#### as_matrix()
- Convert the frame to its Numpy-array representation.

In [10]:
text_train, y_train = df_train['document'].as_matrix(), df_train['label'].as_matrix()

text_test = df_test['document'].as_matrix()
y_test = df_test['label'].as_matrix()

C:\Users\ojkwon\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\Users\ojkwon\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ojkwon\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [11]:
text_train

array(['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
       '너무재밓었다그래서보는것을추천한다', ..., '이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?',
       '청춘 영화의 최고봉.방황과 우울했던 날들의 자화상', '한국 영화 최초로 수간하는 내용이 담긴 영화'],
      dtype=object)

In [12]:
y_train

array([0, 1, 0, ..., 0, 1, 0], dtype=int64)

### Twitter() : 트위터에서 만든 오픈소스 한국어 처리기
- #### 2017년 4.4 버전 이후의 개발은 http://openkoreantext.org 에서 진행
- ### 4가지 기능
     - normalization : 
         - 입니닼ㅋㅋ -> 입니다 ㅋㅋ, 샤릉해 -> 사랑해
     - tokenization
         - 한국어를 처리하는 예시입니다 ㅋㅋ -> 한국어Noun, 를Josa, 처리Noun, 하는Verb, 예시Noun, 입Adjective, 니다Eomi ㅋㅋKoreanParticle
     - stemming
         -  (입니다 -> 이다)
     - phrase extraction 
         - 한국어를 처리하는 예시입니다 ㅋㅋ -> 한국어, 처리, 예시, 처리하는 예시

#### 메소드
- nouns() : 문자열에서 명사만 추출 (명사 추출)
    - ['대한민국', '헌법', '유구', '역사', '전통', '우리', '국민', '운동']
- morphs() : 명사 뿐 아니라 모든 품사의 형태소를 추출 (형태소 추출)
    - ['대한민국', '헌법', '\n\n', '유구', '한', '역사', '와', '전통', '에', '빛나는', '우리', 
    - '대', '한', '국민', '은', '3', '·', '1', '운동', '으로']
- pos() : 모든 형태소 추출 & 품사까지 부착 (품사 부착)
    - [('대한민국', 'Noun'), ('헌법', 'Noun'), ('\n\n', 'Foreign'), ('유구', 'Noun'), ('한', 'Josa'), ('역사', 'Noun'),
 ('와', 'Josa'),
    -  ('전통', 'Noun'), ('에', 'Josa'), ('빛나는', 'Verb'), ('우리', 'Noun'), ('대', 'Modifier'), ('한', 'Modifier'),
    -  ('국민', 'Noun'), ('은', 'Josa'), ('3', 'Number'), ('·', 'Punctuation'), ('1', 'Number'), ('운동', 'Noun'), ('으로', 'Josa')]

In [13]:
def twitter_tokenizer(text):
    return twitter_tag.morphs(text)

In [14]:
twitter_tag = Twitter()

cv = TfidfVectorizer(tokenizer=twitter_tokenizer, min_df=5)   # 적어도 5개 이상의 문서에서 나타난 단어만 사용

C:\Users\ojkwon\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
C:\Users\ojkwon\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


### 학습

### Java 설치
- https://java.com : 다운로드 & 설치

In [15]:
# 일반 PC, 약 8분 소요

lr = LogisticRegression()

x_train = cv.fit_transform(text_train)
x_test = cv.transform(text_test)
result = lr.fit(x_train,y_train)

C:\Users\ojkwon\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### 사용된 단어
- #### vocabulary_ : (단어 : count) 쌍 
- #### get_feature_names() : 특징(단어) 리스트만 반환

In [16]:
cv.vocabulary_

{'아': 12511,
 '더빙': 5223,
 '..': 122,
 '진짜': 18740,
 '짜증나네요': 18863,
 '목소리': 7842,
 '흠': 22296,
 '...': 138,
 '포스터': 20643,
 '보고': 9188,
 '초딩': 19391,
 '영화': 14340,
 '줄': 18248,
 '....': 153,
 '오버': 14510,
 '연기': 14176,
 '조차': 17751,
 '가볍지': 1187,
 '너': 4143,
 '무재': 8058,
 '다그': 4699,
 '래서': 6455,
 '보는것을': 9289,
 '추천': 19517,
 '한': 21206,
 '다': 4690,
 '교도소': 2385,
 '이야기': 15860,
 '구먼': 2427,
 '솔직히': 11375,
 '재미': 16892,
 '는': 4608,
 '없다': 13838,
 '평점': 20615,
 '조정': 17742,
 '사이': 10553,
 '그': 2607,
 '의': 15506,
 '가': 1093,
 '돋보였던': 5450,
 '!': 0,
 '스파이더맨': 11717,
 '에서': 13986,
 '늙어': 4635,
 '했던': 21643,
 '너무나도': 4149,
 '이뻐': 15765,
 '보였다': 9450,
 '막': 6890,
 '걸음': 1837,
 '마': 6765,
 '3': 389,
 '세': 11151,
 '부터': 9893,
 '초등학교': 19389,
 '1': 218,
 '학년': 21192,
 '생인': 10896,
 '8': 501,
 '.': 103,
 'ㅋㅋㅋ': 1007,
 '별': 9136,
 '반개': 8625,
 '도': 5327,
 '아까': 12521,
 '움': 15017,
 '원작': 15132,
 '긴장감': 3052,
 '을': 15477,
 '제대로': 17594,
 '했다': 21628,
 '아깝다': 12545,
 '욕': 14829,
 '나온다': 3705,
 '이응경

In [17]:
len(cv.vocabulary_)

22414

In [18]:
feature_names = cv.get_feature_names()
print(feature_names[3000:3010])

['기억나는게', '기억나는데', '기억나서', '기억나요', '기억나지', '기억난다', '기억남', '기억남는', '기억남는건', '기억상실증']


### 예측

In [19]:
print("훈련 데이터 점수 : ", result.score(x_train, y_train))
print("테스트 데이터 점수 : ", result.score(x_test, y_test))

훈련 데이터 점수 :  0.8816466666666667
테스트 데이터 점수 :  0.8509


### (분석) 고차원 희소행렬 데이터의 경우 로지스틱 회귀를 이용한 분류기의 성능이 우수하다.

## 불용어 처리
- #### 분석 효율을 높이기 위해서 의미 없는 단어를 제거하는 것이 필요
- #### 대표적으로 불용어 처리가 있다. 
    - 문서에 빈번하게 나타나지만 문서의 특징을 나타내거나 분류에 활용되기 어려운 불필요한 단어를 의미 

### 불용어 사전
- #### 인터넷 상에 불용어 리스트를 공개하는 경우가 많이 있으나 
- #### 하나의 불용어 사전을 모든 텍스트 분석에 사용하는 것은 어렵다.
- #### 분석하려는 문서 집합에 따라 적절하게 불용어 사전을 구축하여 활용해야 한다.
    - #### 영어 : scikit-learn의 feature_extraction.text 모듈에 예시를 위한 불용어 사전을 제공
    - #### 한국어 : KoNLPy를 이용


### 한국어 품사 중 조사를 추출하는 코드(예)

In [23]:
from konlpy.tag import Twitter

twitter = Twitter()
word_tags = twitter.pos("텍스트 데이터를 이용하여 불용어 사전을 구축하기 위한 간단 예제", 
                        norm=True, stem=True)

In [24]:
word_tags

[('텍스트', 'Noun'),
 ('데이터', 'Noun'),
 ('를', 'Josa'),
 ('이용', 'Noun'),
 ('하다', 'Verb'),
 ('불', 'Noun'),
 ('용어', 'Noun'),
 ('사전', 'Noun'),
 ('을', 'Josa'),
 ('구축', 'Noun'),
 ('하다', 'Verb'),
 ('위', 'Noun'),
 ('한', 'Josa'),
 ('간단', 'Noun'),
 ('예제', 'Noun')]

In [25]:
stop_words = [word[0] for word in word_tags if word[1]=='Josa']

print(stop_words)

['를', '을', '한']


# 교재 여기까지 ----

# LSTM을 이용한 분석

In [0]:
text_data, y_data = df_data['document'].as_matrix(), df_data['label'].as_matrix()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [0]:
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn import model_selection, metrics
import numpy as np
import pickle
import os.path
import keras.backend.tensorflow_backend as K

# 토큰 파서
def twitter_tokenizer(text):
    return twitter_tag.morphs(text)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [0]:
twitter_tag = Twitter()

In [0]:
cv = TfidfVectorizer(tokenizer=twitter_tokenizer)

In [0]:
!ls X*

X_data.pickle	 X_test_2.pickle  X_train_1.pickle
X_test_1.pickle  X_test.pickle	  X_train_2.pickle


In [0]:
# Tfidf 생성과 저장 - 단 파일이 없을 때만 !!
if not os.path.isfile("X_data.pickle"): 
    print('file does not exists')
    X_data = cv.fit_transform(text_data)
    pickle.dump(X_train, open("X_data.pickle", "wb"))

In [0]:
# 저장된 tfidf vector 데이터 읽기
with open('X_data.pickle', 'rb') as f:
    X_data = pickle.load(f)

In [0]:
!ls -al X*

-rw-rw-r-- 1 hjkim hjkim 24433014 Oct 28 18:39 X_data.pickle
-rw-rw-r-- 1 hjkim hjkim  7782344 Oct 28 19:03 X_test_1.pickle
-rw-rw-r-- 1 hjkim hjkim  5610600 Oct 28 18:56 X_test_2.pickle
-rw-rw-r-- 1 hjkim hjkim  8170628 Oct 28 18:40 X_test.pickle
-rw-rw-r-- 1 hjkim hjkim 23745856 Oct 28 19:02 X_train_1.pickle
-rw-rw-r-- 1 hjkim hjkim 14084384 Oct 28 18:55 X_train_2.pickle


In [0]:
# one-hot encoding
Y_data = np_utils.to_categorical(y_data, 2)

In [0]:
X_train = X_data[:100000]
X_test = X_data[100000:]

Y_train = Y_data[:100000]
Y_test = Y_data[100000:]

In [0]:
max_words = 61070 
nb_classes = 2
batch_size = 1024
nb_epoch = 5

In [0]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
Y_train.shape[1]

(100000, 104160)
(50000, 104160)
(100000, 2)


2

In [0]:
# LSTM 학습을 위한 데이터 재배열 (Time step)
X_train_rnn = X_train.A.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_rnn = X_test.A.reshape((X_test.shape[0], 1, X_test.shape[1]))

print(X_train_rnn.shape)
print(X_test_rnn.shape)

(100000, 1, 104160)
(50000, 1, 104160)


In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

def build_LSTM_model():
    model = Sequential()
    model.add(LSTM(128, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), return_sequences=True))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(LSTM(128))
    model.add(Dropout(0.2))
    model.add(Dense(Y_train.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [0]:
import keras.backend.tensorflow_backend as K
with K.tf.device('/GPU:1'):
    model_lstm = KerasClassifier(
    build_fn=build_LSTM_model, 
    epochs=nb_epoch, 
    batch_size=batch_size)
    
    model_lstm.fit(X_train_rnn, Y_train)

Epoch 1/5
100000/100000 [==============================] - 252s 3ms/step - loss: 0.5463 - acc: 0.7826
Epoch 2/5
100000/100000 [==============================] - 238s 2ms/step - loss: 0.2854 - acc: 0.8830
Epoch 3/5
100000/100000 [==============================] - 239s 2ms/step - loss: 0.2076 - acc: 0.9218
Epoch 4/5
100000/100000 [==============================] - 238s 2ms/step - loss: 0.1604 - acc: 0.9420
Epoch 5/5
100000/100000 [==============================] - 239s 2ms/step - loss: 0.1293 - acc: 0.9543


In [0]:
y = model_lstm.predict(X_train_rnn)
y_train = y_data[:100000]
ac_score = metrics.accuracy_score(y_train, y)
print("훈련 셋 정답률 =", ac_score)

훈련 셋 정답률 = 0.96941


In [0]:
# predict 함수는 예측 결과를 0 or 1로 출력하므로
# 학습과정에서 사용한 Y_train, Y_test 변수로 정확도 측정이 안됨
# Y_train, Y_test는 [0, 1], [1, 0]의 형태로 해당하는 감정 컬럼(class)은 1, 다른 컬럼은 0으로 표시됨
# 초기 y_data에 저장된 값을 그대로 활용하여 정확도를 측정

print("y : ", y)
print("Y_train[0] : ", Y_train)
print("y_train[0] : ", y_data)

y :  [0 1 0 ... 0 1 1]
Y_train[0] :  [[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [0. 1.]]
y_train[0] :  [0 1 0 ... 0 1 0]


In [0]:
y = model_lstm.predict(X_test_rnn)
y_test = y_data[100000:]
ac_score = metrics.accuracy_score(y_test, y)
print("테스트 셋 정답률 =", ac_score)

테스트 셋 정답률 = 0.83784
